#### Start with installing the required library

In [1]:
!pip install snowflake-connector-python[pandas]

     |████████████████████████████████| 13.2 MB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 54.4 MB/s eta 0:00:01
     |████████████████████████████████| 192 kB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 20.7 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 935 kB/s  eta 0:00:01
     |████████████████████████████████| 136 kB 64.1 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 147 kB 47.0 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 4.0.0
    Uninstalling pyarrow-4.0.0:
      Successfully uninstalled pyarrow-4.0.0


In [2]:
import pandas as pd
from snowflake.connector import pandas_tools as sfpd
import snowflake.connector

In [3]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",
                names = ['sex', 'length', 'diameter','height','whole_weight','shucked_weight',
                           'viscera_weight','shell_weight','rings'])

In [7]:
df.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
 conn = snowflake.connector.connect(
                user='DSMONK',
                password='Monk@snowflake007',
                account='oxa27110.us-east-1',
                warehouse='COMPUTE_WH',
                database='sgdb',
                schema='public'
                )

could have used an orm like sqlalchemy, but ended up creating a table using the console. Took adv. of it anyway.

    CREATE TABLE "SGDB"."PUBLIC"."ABALONE_TEST" ("SEX" STRING, "LENGTH" NUMBER (5, 4), "DIAMETER" NUMBER (5, 4), "HEIGHT" NUMBER (5, 4), "WHOLE_WEIGHT" NUMBER (5, 4), "SHUCKED_WEIGHT" NUMBER (5, 4), "VISCERA_WEIGHT" NUMBER (5, 4), "SHELL_WEIGHT" NUMBER (5, 4), "RINGS" INTEGER);

In [14]:
df.columns = [i.upper() for i in df.columns] #update to help with import into snowflake

In [ ]:
df.to_sql('abalone_test',)

In [13]:
success, nchunks, nrows, _ = sfpd.write_pandas(conn, df,"ABALONE_TEST")

In [20]:
print(f"INFO: Write Pandas method reported Success = {success}, with a total number of {nrows} rows written to the table")

INFO: Write Pandas method reported Success = True, with a total number of 4177 rows written to the table


In [22]:
cur = conn.cursor()

In [23]:
sql = "select * from abalone_test"
cur.execute(sql)
res = cur.fetch_pandas_all()

In [24]:
res

,SEX,LENGTH,DIAMETER,HEIGHT,WHOLE_WEIGHT,SHUCKED_WEIGHT,VISCERA_WEIGHT,SHELL_WEIGHT,RINGS
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [25]:
cur.close()

True

In [22]:
conn.close()